In [1]:
import json
import pandas as pd
import cv2
# You may need to restart your runtime prior to this, to let your installation take effect
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog


In [2]:

# def plot_one_box(x, img, color=None, label=None, line_thickness=None, Inverted=False):
# # Plots one bounding box on image img
#     tl = line_thickness or 2 # line thickness

#     c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
#     cv2.rectangle(img, c1, c2, color, thickness=tl)
#     if label:
#         tf = tl # font thickness
#     t_size = cv2.getTextSize(label, 0, fontScale=tl / 2, thickness=tf)[0]
#     if Inverted == True:
#         c1 = c2
#         c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
#     else:
#         c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3

#     cv2.rectangle(img, c1, c2, color, -1) # filled
#     cv2.putText(
#     img,
#     label,
#     (c1[0], c1[1] - 2),
#     0,
#     tl / 2,
#     [225, 255, 255],
#     thickness=tf,
#     lineType=cv2.LINE_AA,
#     )

# # Using readlines()
# file1 = open('train.csv', 'r')
# Lines = file1.readlines()

# count = 0
# # Strips the newline character
# for line in Lines:
#     if count == 0:
#         count += 1
#         continue

#     print(line)
#     file_id_path = line.split(',')[1]

#     print(file_id_path)
#     # open image in cv2
#     img = cv2.imread("images/" + file_id_path)
#     # cv2.imshow("image", img)
#     # cv2.waitKey(20)
#     # cv2.destroyAllWindows()
#     h, w, c = img.shape
#     cat = line.split(',')[2]
#     xmax = int(float(line.split(',')[3])) * 2
#     xmin = int(float(line.split(',')[4])) * 2
#     ymax = int(float(line.split(',')[5])) * 2
#     ymin = int(float(line.split(',')[6])) * 2
#     # plot the box
#     plot_one_box([xmin, ymin, xmax, ymax], img, color=(0, 255, 0), label=cat, line_thickness=2)
#     # save the image
#     # you might need to create the folder "drawn" first!
#     cv2.imwrite("drawn/" + file_id_path, img)
#     print("Line {}: {}".format(count, line.strip()))
#     count += 1
    

In [3]:
def csv_to_coco(file_dir,destination_dir):
    path = file_dir
    save_json_path = destination_dir+'/traincoco.json'
    
    # clmns = ['class','xmin','ymin','xmax','ymax','filename','width','height']
    clmns =  [ "class",	"image_path",	"name",	"xmax",	"xmin",	"ymax",	"ymin"]

    data = pd.read_csv(path, names = clmns, header=None)
    data = data[1:]
    
    mul = 2
    data['xmax'] = data['xmax'].astype(float).astype(int) * mul
    data['xmin'] = data['xmin'].astype(float).astype(int) * mul
    data['ymax'] = data['ymax'].astype(float).astype(int) * mul
    data['ymin'] = data['ymin'].astype(float).astype(int) * mul
    
    images = []
    categories = []
    annotations = []
    data['fileid'] = data['image_path'].astype('category').cat.codes
    # data['categoryid']= pd.Categorical(data['name'],ordered= True).codes
    data['categoryid'] = data['class'].astype(float).astype(int)
    data['annid'] = data.index
    classes = data['name'].unique()
    def image(row):
        image = {}
        image["height"] = 1080
        image["width"] = 1920
        image["id"] = row.fileid
        image["file_name"] = row.image_path
        return image
    
    def category(row):
        category = {}
        category["supercategory"] = 'None'
        category["id"] = row.categoryid 

        category["name"] = row[3]
        return category
    
    def annotation(row):
        annotation = {}
        area = (row.xmax)*(row.ymax)
        annotation["segmentation"] = []
        annotation["iscrowd"] = 0
        annotation["area"] = area
        annotation["image_id"] = row.fileid
        annotation["bbox"] = [row.xmin, row.ymin, row.xmax +row.xmin,row.ymax+row.ymin ]
        annotation["bbox_mode"] = BoxMode.XYXY_ABS
        annotation["category_id"] = row.categoryid 
        annotation["id"] = row.annid
        return annotation
    
    for row in data.itertuples():
        annotations.append(annotation(row))
    imagedf = data.drop_duplicates(subset=['fileid']).sort_values(by='fileid')
    for row in imagedf.itertuples():
        images.append(image(row))
    catdf = data.drop_duplicates(subset=['categoryid']).sort_values(by='categoryid')
    for row in catdf.itertuples():
        categories.append(category(row))

    data_coco = {}
    data_coco["images"] = images
    data_coco["categories"] = categories
    data_coco["annotations"] = annotations
    json.dump(data_coco, open(save_json_path, "w"), indent=4)
    return classes


In [4]:
file_dir = 'train.csv'
destination_dir = 'coco'
classes = csv_to_coco(file_dir,destination_dir)
print(classes)

['GARBAGE' 'BAD_BILLBOARD' 'SAND_ON_ROAD' 'GRAFFITI' 'POTHOLES'
 'CLUTTER_SIDEWALK' 'CONSTRUCTION_ROAD' 'BROKEN_SIGNAGE' 'UNKEPT_FACADE'
 'FADED_SIGNAGE' 'BAD_STREETLIGHT']


In [5]:
def get_board_dicts(imgdir):
    json_file = imgdir+"/traincoco.json" #Fetch the json file
    with open(json_file) as f:
        dataset_dicts = json.load(f)
    # for i in dataset_dicts:
    #     filename = i["file_name"] 
    #     i["file_name"] = imgdir+"/"+filename 
    #     for j in i["annotations"]:
    #         j["bbox_mode"] = BoxMode.XYWH_ABS #Setting the required Box Mode
    #         j["category_id"] = int(j["category_id"])
    return dataset_dicts

# for d in ["train", "val"]:
dataset = json.load(open("coco/traincoco.json"))
from detectron2.data.datasets import register_coco_instances
register_coco_instances("smartcity", {}, "./coco/traincoco.json", "./images/")

In [6]:
metadata = MetadataCatalog.get("smartcity")
dataset_dicts = DatasetCatalog.get("smartcity")
# print(dataset_dicts), print(metadata)

WARNING [01/27 16:56:01 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[01/27 16:56:01 d2.data.datasets.coco]: Loaded 7874 images in COCO format from ./coco/traincoco.json


In [7]:
import random
from detectron2.utils.visualizer import Visualizer

for d in random.sample(dataset_dicts, 10):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    frame = vis.get_image()[:, :, ::-1]
    cv2.imshow("",frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [9]:

from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")) #Get the basic model configuration from the model zoo 
cfg.DATASETS.TRAIN = ("smartcity",) #Passing the Train and Validation sets
cfg.MODEL.DEVICE = "cuda"
cfg.DATALOADER.NUM_WORKERS = 4 # Number of data loading threads
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4 # Number of images per batch across all machines.
cfg.SOLVER.BASE_LR = 0.0125  # pick a good LearningRate
cfg.SOLVER.MAX_ITER = 1500  #No. of iterations   
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256  
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 11 # No. of classes 
# cfg.TEST.EVAL_PERIOD = 500 # No. of iterations after which the Validation Set is evaluated. 
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()


SyntaxError: trailing comma not allowed without surrounding parentheses (3510773385.py, line 1)